In [6]:
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set_style('darkgrid')

import pandas as pd
import numpy as np
import os

import h5py

In [7]:
def gen_plots(path, file_name):
#     path = '.'
#     file_name = 'Session_02_Oct_2019_07_24_51_Set1to50.mat'

    f = h5py.File(os.path.join(path, file_name))
    f.keys()
    sess_data = f['sess_data']

    frame_size = np.shape(sess_data['plotData'])[1]

    frame = pd.Series()
    look_type=pd.Series()
    rng = pd.Series()
    speed = pd.Series()
    az=pd.Series()
    el=pd.Series()
    amp=pd.Series()
    snr=pd.Series()

    for frame_idx in range(0, frame_size):
    #     look_type = look_type.append(pd.Series(f[sess_data['plotData'][0,frame_idx]]['look'][()][0,0]), ignore_index=True)

        look_type_temp = f[sess_data['plotData'][0,frame_idx]]['look'][()][0,0]
        frame_str = ((f[sess_data['RawDataFile'][frame_idx,0]][()]-48).ravel())[22:-4]
        frame_temp = int(str(frame_str)[1:-1].replace(" ", ""))

        num_rdop_det = np.shape(f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'])[0]

        for rdop_det_idx in range(0, num_rdop_det):
            rng_temp = pd.Series(f[f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'][rdop_det_idx,0]]['range'][()][0,:])
            speed_temp = pd.Series(f[f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'][rdop_det_idx,0]]['range_rate'][()][0,:])
            az_temp= pd.Series(f[f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'][rdop_det_idx,0]]['az'][()][0,:])
            el_temp= pd.Series(f[f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'][rdop_det_idx,0]]['el'][()][0,:])
            amp_temp= pd.Series(f[f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'][rdop_det_idx,0]]['amp'][()][0,:])
            snr_temp= pd.Series(f[f[sess_data['plotData'][0,frame_idx]]['afData']['af_output'][rdop_det_idx,0]]['SNR'][()][0,:])

            rng = rng.append(rng_temp, ignore_index=True)
            speed = speed.append(speed_temp, ignore_index=True)
            az = az.append(az_temp, ignore_index=True)
            el = el.append(el_temp, ignore_index=True)
            amp = amp.append(amp_temp, ignore_index=True)
            snr = snr.append(snr_temp, ignore_index=True)

            look_type=look_type.append(pd.Series(np.ones_like(rng_temp)*look_type_temp), ignore_index=True)
            frame=frame.append(pd.Series(np.ones_like(rng_temp)*frame_temp), ignore_index=True)

            det_list = pd.DataFrame()

    det_list['Frame'] = frame
    det_list['LookType'] = look_type
    det_list['Range'] = rng
    det_list['Speed'] = speed
    det_list['Azimuth'] = az
    det_list['Elevation'] = el
    det_list['Amp'] = amp
    det_list['SNR'] = snr

    det_list['Target_loc_x'] = det_list['Range']*np.sin(det_list['Azimuth']/180*np.pi)*np.cos(
        det_list['Elevation']/180*np.pi)
    det_list['Target_loc_y'] = det_list['Range']*np.cos(det_list['Azimuth']/180*np.pi)*np.cos(
        det_list['Elevation']/180*np.pi)
    det_list['Target_rng_y'] = det_list['Range']*np.cos(det_list['Azimuth']/180*np.pi)*np.cos(
        det_list['Elevation']/180*np.pi)
    det_list['Target_loc_z'] = det_list['Range'] * \
        np.sin(det_list['Elevation']/180*np.pi)

    data = []
    frame_list = det_list['Frame'].unique()
    for frame_idx in frame_list:
        filtered_list = det_list[det_list['Frame'] == frame_idx]
        filtered_list = filtered_list.reset_index()
        x = filtered_list['Target_loc_x']
        y = filtered_list['Target_loc_y']
        z = filtered_list['Target_loc_z']
        amp = 20*np.log10(filtered_list['Amp'])
        # rcs = filtered_list['RCS']
        frame = filtered_list['Frame']
        snr = 20*np.log10(filtered_list['SNR'])
        az = filtered_list['Azimuth']
        el = filtered_list['Elevation']
        # unfolding_fail = filtered_list['Unfolding_fail']
        # af_type = filtered_list['AF Type']
        rng = filtered_list['Range']
        speed = filtered_list['Speed']

        hover = []
        # vehicle_hover = []
        for idx, var in enumerate(frame.to_list()):
            hover.append('Frame: '+str(int(var))+'<br>' +
                         'Amp: '+'{:.2f}'.format(amp[idx])+'dB<br>' +
                         #                  'RCS: ' + '{:.2f}'.format(rcs[idx])+'dB<br>' +
                         'SNR: ' + '{:.2f}'.format(snr[idx])+'dB<br>' +
                         'Az: ' + '{:.2f}'.format(az[idx])+'deg<br>' +
                         'El: ' + '{:.2f}'.format(el[idx])+'deg<br>' +
                         'Range: ' + '{:.2f}'.format(rng[idx])+'m<br>' +
                         'Speed: ' + '{:.2f}'.format(speed[idx])+'m/s<br>')
    #         vehicle_hover.append('Frame: '+str(var)+'<br>')

        det_map = go.Scatter3d(
            x=x,
            y=y,
            z=z,
            text=hover,
            hovertemplate='%{text}'+'Lateral: %{x:.2f} m<br>' +
            'Longitudinal: %{y:.2f} m<br>'+'Height: %{z:.2f} m<br>',
            mode='markers',
            name='Frame: '+str(int(frame_idx)),
            marker=dict(
                size=3,
                color=speed,                # set color to an array/list of desired values
                colorscale='Jet',   # choose a colorscale
                opacity=0.5,
                colorbar=dict(
                    title='Speed (m/s)',
                ),
                cmin=-20,
                cmax=20,
            )
        )

        data.append(det_map)

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=0, y=-1.5, z=1.8),
    )

    layout = go.Layout(
        title=file_name,
        scene=dict(xaxis=dict(title='Lateral (m)'),
                   yaxis=dict(title='Longitudinal (m)'),
                   zaxis=dict(title='Height (m)'),
                   camera=camera,
                   aspectmode='data'),
        margin=dict(l=0, r=0, b=0, t=40),
        height=800,
    )
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(legend_orientation="h")
#     fig.show()
    fig.write_html(file_name[:-4]+'.html')

In [8]:
path = './GOT'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.mat' in file:
            files.append(file)

In [9]:
for f in files:
    gen_plots(path, f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning:

The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.



ValueError: Index (19) out of range (0-18)